In [5]:
import glob

from src.create_annotations import *
from src.carla_utils import *

# Label ids of the dataset
# category_ids = {
#     "outlier": 0,
#     "window": 1,
#     "wall": 2,
#     "balcony": 3,
#     "door": 4,
#     "roof": 5,
#     "sky": 6,
#     "shop": 7,
#     "chimney": 8
# }

# # Define which colors match which categories in the images
# category_colors = {
#     "(0, 0, 0)": 0, # Outlier
#     "(255, 0, 0)": 1, # Window
#     "(255, 255, 0)": 2, # Wall
#     "(128, 0, 255)": 3, # Balcony
#     "(255, 128, 0)": 4, # Door
#     "(0, 0, 255)": 5, # Roof
#     "(128, 255, 255)": 6, # Sky
#     "(0, 255, 0)": 7, # Shop
#     "(128, 128, 128)": 8 # Chimney
# }

category_ids = {
    0: "Unlabeled",
    1: "Roads",
    2: "SideWalks",
    3: "Building",
    4: "Wall",
    5: "Fence",
    6: "Pole",
    7: "TrafficLight",
    8: "TrafficSign",
    9: "Vegetation",
    10: "Terrain",
    11: "Sky",
    12: "Pedestrian",
    13: "Rider",
    14: "Car",
    15: "Truck",
    16: "Bus",
    17: "Train",
    18: "Motorcycle",
    19: "Bicycle",
    20: "Static",
    21: "Dynamic",
    22: "Other",
    23: "Water",
    24: "RoadLine",
    25: "Ground",
    26: "Bridge",
    27: "RailTrack",
    28: "GuardRail"
}

# category_colors = {

# }

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
multipolygon_ids = [2, 5, 6]

# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".jpg"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, w, h)
        for color, sub_mask in sub_masks.items():
            print("this is the color:", color, "and type: ",type(color))

            r = split_rgb(color)

            category_id = r

            print("category id is:",category_id)


            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                                
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id

if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()

    mask_path = "dataset/carla_mask/" 

    coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
    coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    with open("output/carla.json","w") as outfile:
        json.dump(coco_format, outfile)
    
    print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))
    
    # for keyword in ["train", "val"]:
    #     mask_path = "dataset/{}_mask/".format(keyword)
        
    #     # Create category section
    #     coco_format["categories"] = create_category_annotation(category_ids)
    
    #     # Create images and annotations sections
    #     coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    #     with open("output/{}.json".format(keyword),"w") as outfile:
    #         json.dump(coco_format, outfile)
        
    #     print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))


this is the color: (3, 242, 134) and type:  <class 'str'>
rgb values: ['3', ' 242', ' 134'] type is:  <class 'list'>
category id is: 3
this is the color: (3, 66, 135) and type:  <class 'str'>
rgb values: ['3', ' 66', ' 135'] type is:  <class 'list'>
category id is: 3
this is the color: (3, 69, 135) and type:  <class 'str'>
rgb values: ['3', ' 69', ' 135'] type is:  <class 'list'>
category id is: 3
this is the color: (3, 72, 135) and type:  <class 'str'>
rgb values: ['3', ' 72', ' 135'] type is:  <class 'list'>
category id is: 3
this is the color: (15, 191, 213) and type:  <class 'str'>
rgb values: ['15', ' 191', ' 213'] type is:  <class 'list'>
category id is: 15
this is the color: (1, 157, 115) and type:  <class 'str'>
rgb values: ['1', ' 157', ' 115'] type is:  <class 'list'>
category id is: 1
this is the color: (24, 225, 114) and type:  <class 'str'>
rgb values: ['24', ' 225', ' 114'] type is:  <class 'list'>
category id is: 24
this is the color: (14, 210, 157) and type:  <class 'st